In [29]:
"""
CELL 1: SETUP (WITH TIMING)
Description: Load configuration and all VAANI functions with timing
"""

import time

print("="*60)
print("🎮 VAANI PLAYGROUND - LOADING")
print("="*60)

# Time Config Loading
config_start = time.time()
%run vyapar_config.ipynb
config_time = time.time() - config_start
print(f"✅ Config loaded: {config_time*1000:.0f} ms")

# Time Prompts Loading
prompts_start = time.time()
%run vaani_prompts.ipynb
prompts_time = time.time() - prompts_start
print(f"✅ Prompts loaded: {prompts_time*1000:.0f} ms")

# Time Functions Loading
functions_start = time.time()
%run vaani_functions.ipynb
functions_time = time.time() - functions_start
print(f"✅ Functions loaded: {functions_time*1000:.0f} ms")

# Summary
total_time = config_time + prompts_time + functions_time
print("\n" + "="*60)
print(f"⏱️ TOTAL LOAD TIME: {total_time*1000:.0f} ms ({total_time:.2f}s)")
print("="*60)
print(f"   Config:    {config_time*1000:>6.0f} ms ({config_time/total_time*100:>4.1f}%)")
print(f"   Prompts:   {prompts_time*1000:>6.0f} ms ({prompts_time/total_time*100:>4.1f}%)")
print(f"   Functions: {functions_time*1000:>6.0f} ms ({functions_time/total_time*100:>4.1f}%)")
print("="*60)
print("✅ Ready to test!")

🎮 VAANI PLAYGROUND - LOADING
✅ Libraries imported successfully
✅ Master Registry Link configured
📍 Link: https://docs.google.com/spreadsheets/d/e/2PACX-1vQdOVYDNLuMG...

🔄 Testing registry load...
✅ Registry loaded successfully: 11 items found

📋 Available items in registry:
  1. default_model
  2. openai_api_key
  3. anthropic_api_key
  4. master_registry_link
  5. usage_data
  6. dropoff_analysis
  7. user_research
  8. excel_db_1000_items
  9. claude_haiku
  10. vaani_test_cases
  11. consolidated_categories

🧪 Testing get_item() function:

1. Testing with 'default_model':
✅ Registry loaded successfully: 11 items found
✅ Retrieved 'default_model'
   Result: {'model': 'claude-sonnet-4-20250514', 'temperature': 0.3, 'max_tokens': 1000, 'provider': 'anthropic'}

2. Testing with non-existent item:
✅ Registry loaded successfully: 11 items found
❌ Item 'this_does_not_exist' not found in registry
💡 Available items: default_model, openai_api_key, anthropic_api_key, master_registry_link, usa

In [18]:
"""
CELL 2: QUICK SINGLE TEST
"""

timer = CellTimer()

# ============ MODIFY THESE ============
test_input = "chai samosa 140 rupees"
transaction_type = None  # Optional: 'expense', 'sale', etc.
# ======================================

print("="*60)
print(f"📝 Testing: '{test_input}'")
print("="*60)

result = route_with_intent(test_input, transaction_type=transaction_type)

print(f"\n📊 RESULT:")
print(f"Status: {result.get('status')}")

if result.get('status') == 'success':
    print(f"Transaction Type: {result.get('transaction_type')}")
    
    print(f"\n✅ Extracted Data:")
    extraction = result.get('extraction', {})
    for key, value in extraction.items():
        if key not in ['raw_response', 'transaction_type']:
            print(f"   {key}: {value}")
    
    if 'time_taken' in extraction:
        print(f"\n⏱️ API Time: {extraction['time_taken']*1000:.0f} ms")

elif result.get('status') == 'not_relevant':
    print(f"⚠️ {result.get('message')}")
else:
    print(f"❌ Error: {result.get('error')}")

timer.print_summary("TOTAL")

📝 Testing: 'chai samosa 140 rupees'
🔍 Running intent detection...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
📊 Extracting expense data...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized

📊 RESULT:
Status: success
Transaction Type: expense

✅ Extracted Data:
   amount: 140
   item: chai and samosa
   category: Food & Beverages
   model_used: claude-sonnet-4-20250514
   time_taken: 4.82

⏱️ API Time: 4820 ms

⏱️ TOTAL TIME: 14994 ms (14.99s)


In [25]:
"""
CELL 2B: DETAILED TIMING BREAKDOWN
Description: Test with complex input and show timing for each AI agent
"""

import time

# ============ MODIFY THIS ============
test_input = "Sharma ji se vegetables 450 rupees liye aur unhe 50 rupees tip diya delivery ke liye total 500 rupees cash mein"
transaction_type = None  # Set to skip intent detection
# =====================================

print("="*60)
print(f"🔬 DETAILED TIMING BREAKDOWN")
print("="*60)
print(f"Input: '{test_input}'")
print("="*60)

# Initialize timing variables
total_start = time.time()
intent_time = 0
extraction_time = 0
detected_type = None
skip_extraction = False

result = {
    'input': test_input,
    'intent_detection_skipped': transaction_type is not None
}

# Step 1: Intent Detection
if transaction_type is None:
    print("\n⏱️ STEP 1: Intent Detection")
    print("-" * 60)
    
    intent_start = time.time()
    intent_result = agent_intent_detector(test_input)
    intent_time = time.time() - intent_start
    
    print(f"✅ Completed in {intent_time*1000:.0f} ms")
    
    if 'error' in intent_result:
        print(f"❌ Error: {intent_result['error']}")
        skip_extraction = True
    else:
        print(f"   Relevant: {intent_result.get('is_relevant')}")
        print(f"   Type: {intent_result.get('transaction_type')}")
        print(f"   Confidence: {intent_result.get('confidence')}")
        
        result['intent'] = intent_result
        detected_type = intent_result.get('transaction_type')
        
        # Handle special cases
        if detected_type == 'greeting':
            result['status'] = 'greeting'
            print(f"\n💬 Greeting detected - showing friendly response")
            skip_extraction = True
        
        elif detected_type == 'not_relevant' or not intent_result.get('is_relevant'):
            result['status'] = 'not_relevant'
            print(f"\n⚠️ Not relevant for business transactions")
            skip_extraction = True
else:
    print(f"\n⏭️ Skipping intent detection (type: {transaction_type})")
    detected_type = transaction_type

# Step 2: Transaction Extraction (only if not greeting/not_relevant)
if not skip_extraction and detected_type:
    print(f"\n⏱️ STEP 2: Transaction Extraction ({detected_type})")
    print("-" * 60)
    
    extraction_start = time.time()
    extraction_result = agent_transaction_extractor(test_input, detected_type)
    extraction_time = time.time() - extraction_start
    
    print(f"✅ Completed in {extraction_time*1000:.0f} ms")
    
    if 'error' in extraction_result:
        print(f"❌ Error: {extraction_result['error']}")
    else:
        print(f"\n📊 Extracted Data:")
        for key, value in extraction_result.items():
            if key not in ['raw_response', 'transaction_type']:
                print(f"   {key}: {value}")
    
    result['extraction'] = extraction_result
    result['status'] = 'success'
    result['transaction_type'] = detected_type

# Calculate total time
total_time = time.time() - total_start
overhead_time = total_time - intent_time - extraction_time

# Detailed Breakdown
print("\n" + "="*60)
print("⏱️ DETAILED TIMING BREAKDOWN")
print("="*60)

if intent_time > 0:
    print(f"Intent Detection:  {intent_time*1000:>7.0f} ms ({intent_time/total_time*100:>5.1f}%)")

if extraction_time > 0:
    print(f"Extraction:        {extraction_time*1000:>7.0f} ms ({extraction_time/total_time*100:>5.1f}%)")

print(f"Overhead:          {overhead_time*1000:>7.0f} ms ({overhead_time/total_time*100:>5.1f}%)")
print("-" * 60)
print(f"TOTAL TIME:        {total_time*1000:>7.0f} ms (100.0%)")
print("="*60)

# API Time Breakdown
print(f"\n🔍 API Processing Time:")
if intent_time > 0:
    print(f"   Intent API:     {intent_time*1000:.0f} ms")
if extraction_time > 0:
    print(f"   Extraction API: {extraction_time*1000:.0f} ms")
    print(f"   Combined API:   {(intent_time + extraction_time)*1000:.0f} ms")

print("\n💡 Note: Overhead includes Python execution, JSON parsing, etc.")

# Show result status
if result.get('status') == 'greeting':
    print("\n💬 This was a greeting - no extraction needed")
elif result.get('status') == 'not_relevant':
    print("\n⚠️ This was not relevant - no extraction needed")
elif result.get('status') == 'success':
    print(f"\n✅ Successfully extracted {detected_type} data")

🔬 DETAILED TIMING BREAKDOWN
Input: 'Sharma ji se vegetables 450 rupees liye aur unhe 50 rupees tip diya delivery ke liye total 500 rupees cash mein'

⏱️ STEP 1: Intent Detection
------------------------------------------------------------
✅ Registry loaded successfully: 10 items found
✅ Registry loaded successfully: 10 items found
✅ Anthropic client initialized
✅ Completed in 9083 ms
   Relevant: True
   Type: expense
   Confidence: 0.9

⏱️ STEP 2: Transaction Extraction (expense)
------------------------------------------------------------
✅ Registry loaded successfully: 10 items found
✅ Registry loaded successfully: 10 items found
✅ Anthropic client initialized
✅ Completed in 8774 ms

📊 Extracted Data:
   amount: 500
   item: Vegetables with delivery tip
   category: Food & Beverages
   model_used: claude-sonnet-4-20250514
   time_taken: 6.32

⏱️ DETAILED TIMING BREAKDOWN
Intent Detection:     9083 ms ( 50.9%)
Extraction:           8774 ms ( 49.1%)
Overhead:                2 ms (  0.

In [30]:
"""
CELL 2C: VIEW CONSOLIDATED CATEGORIES
Description: Load and display categories from registry
"""

print("="*60)
print("📊 CONSOLIDATED EXPENSE CATEGORIES")
print("="*60)

# Load categories
df_categories = load_categories_data()

if df_categories is not None:
    print(f"\n✅ Loaded {len(df_categories)} categories from registry\n")
    
    # Display top 20
    print("TOP 20 MOST USED CATEGORIES:")
    print("-"*60)
    
    for idx, row in df_categories.head(20).iterrows():
        rank = row['#']
        category = row['Consolidated Category']
        count = row['Total Count']
        notes = row.get('Notes', '')
        
        print(f"{rank:2}. {category:25} ({count:>12} uses)")
        if notes:
            print(f"    → {notes}")
    
    print("\n" + "="*60)
    
    # Show distribution
    print("\n📈 CATEGORY DISTRIBUTION:")
    print("-"*60)
    
    total_count = df_categories['Total Count'].apply(lambda x: int(str(x).replace(',', ''))).sum()
    
    print(f"Top 10 categories: {df_categories.head(10)['Total Count'].apply(lambda x: int(str(x).replace(',', ''))).sum() / total_count * 100:.1f}% of all expenses")
    print(f"Top 20 categories: {df_categories.head(20)['Total Count'].apply(lambda x: int(str(x).replace(',', ''))).sum() / total_count * 100:.1f}% of all expenses")
    print(f"Top 30 categories: {df_categories.head(30)['Total Count'].apply(lambda x: int(str(x).replace(',', ''))).sum() / total_count * 100:.1f}% of all expenses")
    
    print("\n" + "="*60)
    
    # Show which categories handle common items
    print("\n🔍 CATEGORY MAPPING EXAMPLES:")
    print("-"*60)
    
    examples = {
        "chai 50 rupees": "Tea Coffee",
        "petrol 500": "Fuel",
        "taxi 200": "Transport",
        "salary 15000": "Salary",
        "electricity bill 2500": "Electricity",
        "internet 1000": "Internet",
        "office supplies 300": "Office Expenses",
        "courier charges 150": "Freight Courier"
    }
    
    for example, expected_category in examples.items():
        print(f"'{example:30}' → {expected_category}")
    
    print("\n" + "="*60)
else:
    print("❌ Could not load categories")

📊 CONSOLIDATED EXPENSE CATEGORIES
✅ Registry loaded successfully: 11 items found
✅ Loaded 50 categories

✅ Loaded 50 categories from registry

TOP 20 MOST USED CATEGORIES:
------------------------------------------------------------
 1. Indirect Expenses         (     1388006 uses)
    → Most used category
 2. Petrol                    (      330413 uses)
    → Merged: Petrol (211,957) + Fuil Exp A/c (118,508) + Diesel
 3. Direct Expenses           (      299989 uses)
    → Second most common expense type
 4. Salary                    (      239200 uses)
    → Includes wages and salary variations
 5. Transport                 (      192905 uses)
    → Transportation and travel related
 6. Tea Coffee                (      125072 uses)
    → Merged tea, coffee, chai variations
 7. Bank Charges              (      104635 uses)
    → BANK CHARGES (55,800) + Bank Charges (48,835)
 8. Rent                      (       78571 uses)
    → Property rental expenses
 9. Electricity               (

In [31]:
"""
CELL 2D: TEST CATEGORY ACCURACY
Description: Test if expense extraction uses correct categories
"""

# Test cases with expected categories
test_cases = [
    {"input": "chai 50 rupees", "expected": "Tea Coffee"},
    {"input": "Diesel fuel 500", "expected": "Petrol"},
    {"input": "taxi ke liye 200 diye", "expected": "Transport"},
    {"input": "electricity bill 2500 rupees", "expected": "Electricity"},
    {"input": "internet ka bill 1000", "expected": "Internet"},
    {"input": "courier charges 150", "expected": "Freight Courier"},
    {"input": "office stationery 300", "expected": "Stationery"},
    {"input": "printing 500 rupees", "expected": "Printing"},
    {"input": "mobile recharge 200", "expected": "Phone Mobile"},
    {"input": "GST payment 5000", "expected": "GST"},
]

print("="*60)
print("🧪 TESTING CATEGORY ACCURACY")
print("="*60)
print(f"Testing {len(test_cases)} cases...\n")

results = []

for test in test_cases:
    test_input = test['input']
    expected_cat = test['expected']
    
    # Extract
    result = route_with_intent(test_input, transaction_type='expense')
    
    if result.get('status') == 'success':
        extracted = result['extraction']
        detected_cat = extracted.get('category', 'N/A')
        
        match = (detected_cat == expected_cat)
        icon = "✅" if match else "❌"
        
        print(f"{icon} '{test_input:35}' → {detected_cat:20} (expected: {expected_cat})")
        
        results.append({
            'input': test_input,
            'expected': expected_cat,
            'detected': detected_cat,
            'correct': match
        })
    else:
        print(f"❌ '{test_input:35}' → ERROR")

# Summary
print("\n" + "="*60)
correct = sum(1 for r in results if r['correct'])
print(f"✅ Accuracy: {correct}/{len(results)} ({correct/len(results)*100:.1f}%)")
print("="*60)

🧪 TESTING CATEGORY ACCURACY
Testing 10 cases...

⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 11 items found
✅ Registry loaded successfully: 11 items found
✅ Anthropic client initialized
✅ 'chai 50 rupees                     ' → Tea Coffee           (expected: Tea Coffee)
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 11 items found
✅ Registry loaded successfully: 11 items found
✅ Anthropic client initialized
✅ 'Diesel fuel 500                    ' → Petrol               (expected: Petrol)
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 11 items found
✅ Registry loaded successfully: 11 items found
✅ Anthropic client initialized
✅ 'taxi ke liye 200 diye              ' → Transport            (expected: Transport)
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded succe

In [19]:
"""
CELL 3: ENHANCED BATCH TEST
Description: Test multiple inputs across Sonnet and Haiku
"""

timer = CellTimer()

# ============ TEST CONFIGURATION ============
test_cases = [
    # Expenses
    {"input": "chai samosa 140 rupees", "type": "expense"},
    {"input": "petrol 500 rupees", "type": "expense"},
    {"input": "taxi ke liye 200 diye", "type": "expense"},
    {"input": "delivery charges 50", "type": "expense"},
    
    # Sales
    {"input": "Sharma ji bought 5kg rice for 250 rupees", "type": "sale"},
    {"input": "sold vegetables to Ramesh 300 rupees", "type": "sale"},
    
    # Payments In
    {"input": "Mishra aunty ne 500 ka udhar chukaya", "type": "payment_in"},
    {"input": "received 2000 from Kumar", "type": "payment_in"},
    
    # Payments Out
    {"input": "paid 1000 rupees to supplier", "type": "payment_out"},
    {"input": "Gupta ji ko 5000 diye", "type": "payment_out"},
    
    # Not Relevant
    {"input": "what's the weather today?", "type": None},
    {"input": "hello how are you", "type": None},
]

models = [
    ('claude-sonnet-4-5-20250929', 'Sonnet'),
    ('claude-haiku-4-5-20251001', 'Haiku'),
]

skip_intent_modes = [
    (False, 'Auto-detect'),
    (True, 'Skip Intent'),
]
# ==========================================

print("="*60)
print(f"🧪 COMPREHENSIVE BATCH TEST")
print(f"Test Cases: {len(test_cases)}")
print(f"Models: {len(models)}")
print(f"Modes: {len(skip_intent_modes)}")
print(f"Total Tests: {len(test_cases) * len(models) * len(skip_intent_modes)}")
print("="*60)

all_results = []

for case_idx, test_case in enumerate(test_cases, 1):
    test_input = test_case['input']
    expected_type = test_case['type']
    
    print(f"\n[{case_idx}/{len(test_cases)}] '{test_input[:45]}...'")
    
    for model_id, model_name in models:
        for skip_intent, mode_name in skip_intent_modes:
            
            if skip_intent and expected_type:
                tx_type_param = expected_type
            else:
                tx_type_param = None
            
            result = route_with_intent(test_input, transaction_type=tx_type_param)
            
            row = {
                'input': test_input[:40],
                'expected_type': expected_type or 'not_relevant',
                'model': model_name,
                'mode': mode_name,
                'status': result.get('status'),
                'detected_type': result.get('transaction_type', 'N/A'),
                'time_ms': 'N/A',
                'amount': 'N/A',
            }
            
            if result.get('status') == 'success':
                extraction = result.get('extraction', {})
                row['time_ms'] = int(extraction.get('time_taken', 0) * 1000)
                row['amount'] = extraction.get('amount', 'N/A')
                status_icon = '✅'
            elif result.get('status') == 'not_relevant':
                status_icon = '⚠️'
            else:
                status_icon = '❌'
                row['amount'] = 'ERROR'
            
            all_results.append(row)
            print(f"   {status_icon} {model_name:6} | {mode_name:12} | {row['time_ms']:>6} ms")

df = pd.DataFrame(all_results)

# Summary by Model
print("\n" + "="*60)
print("📊 SUMMARY BY MODEL")
print("="*60)

for model_name in ['Sonnet', 'Haiku']:
    model_df = df[df['model'] == model_name]
    success_count = len(model_df[model_df['status'] == 'success'])
    
    print(f"\n🔹 {model_name}:")
    print(f"   Success: {success_count}/{len(model_df)} ({success_count/len(model_df)*100:.1f}%)")
    
    times = pd.to_numeric(model_df[model_df['time_ms'] != 'N/A']['time_ms'], errors='coerce')
    if len(times) > 0:
        print(f"   Avg Time: {times.mean():.0f} ms")
        print(f"   Min Time: {times.min():.0f} ms")
        print(f"   Max Time: {times.max():.0f} ms")

# Summary by Mode
print("\n" + "="*60)
print("📊 SUMMARY BY MODE")
print("="*60)

for mode_name in ['Auto-detect', 'Skip Intent']:
    mode_df = df[df['mode'] == mode_name]
    success_count = len(mode_df[mode_df['status'] == 'success'])
    
    print(f"\n🔹 {mode_name}:")
    print(f"   Success: {success_count}/{len(mode_df)} ({success_count/len(mode_df)*100:.1f}%)")
    
    times = pd.to_numeric(mode_df[mode_df['time_ms'] != 'N/A']['time_ms'], errors='coerce')
    if len(times) > 0:
        print(f"   Avg Time: {times.mean():.0f} ms")

# Overall accuracy
print("\n" + "="*60)
print("📈 OVERALL ACCURACY")
print("="*60)

correct = len(df[(df['expected_type'] == df['detected_type']) | 
                  ((df['expected_type'] == 'not_relevant') & (df['status'] == 'not_relevant'))])
total = len(df)
print(f"Correct Type Detection: {correct}/{total} ({correct/total*100:.1f}%)")

timer.print_summary("TOTAL BATCH TEST")
print(f"⏱️ Avg per test: {timer.elapsed()/len(all_results)*1000:.0f} ms")
print("="*60)

🧪 COMPREHENSIVE BATCH TEST
Test Cases: 12
Models: 2
Modes: 2
Total Tests: 48

[1/12] 'chai samosa 140 rupees...'
🔍 Running intent detection...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
📊 Extracting expense data...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
   ✅ Sonnet | Auto-detect  |   6370 ms
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
   ✅ Sonnet | Skip Intent  |   5740 ms
🔍 Running intent detection...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
📊 Extracting expense data...
✅ Registry loaded successfully: 9 items found
✅ Registry loaded successfully: 9 items found
✅ Anthropic client initialized
 

In [20]:
"""
CELL 4: VIEW PROMPTS & FIELDS
"""

# ============ MODIFY THIS ============
what_to_view = 'fields'  # Options: 'fields', 'prompts', 'expense', 'sale', 'intent', etc.
# =====================================

print("\n")

if what_to_view == 'fields':
    view_transaction_fields()
elif what_to_view == 'prompts':
    view_all_prompts()
else:
    view_prompt(what_to_view)



📊 TRANSACTION FIELD REQUIREMENTS

🔹 EXPENSE
   Description: Recording business expenses
   ✅ Necessary: amount, item
   📝 Additional: category, date, payment_type, notes

🔹 SALE
   Description: Recording sales/invoices
   ✅ Necessary: customer_name, amount, items
   📝 Additional: payment_type, date, notes, invoice_number

🔹 PURCHASE
   Description: Recording purchases from suppliers
   ✅ Necessary: supplier_name, amount, items
   📝 Additional: payment_type, date, notes, inventory_update

🔹 PAYMENT_IN
   Description: Money received from customers
   ✅ Necessary: payer_name, amount
   📝 Additional: payment_type, date, notes, invoice_reference

🔹 PAYMENT_OUT
   Description: Money paid to vendors/suppliers
   ✅ Necessary: payee_name, amount
   📝 Additional: payment_type, date, notes, invoice_reference



In [21]:
"""
CELL 5: CUSTOM TEST AREA
Description: Free space for experiments
"""

timer = CellTimer()

# Your custom code here
# Example:
# result = route_with_intent("your test input")
# print(result)

print("💡 Use this cell for custom experiments")

timer.print_summary("CUSTOM TEST")

💡 Use this cell for custom experiments

⏱️ CUSTOM TEST TIME: 0 ms (0.00s)


In [32]:
"""
CELL: PRODUCTION CODE COMPARISON & ANALYSIS
Description: Understand production implementation and compare with notebook development
"""

import json
import pandas as pd

print("="*70)
print("🏭 VAANI PRODUCTION CODE ANALYSIS")
print("="*70)

# ============================================================================
# PART 1: Architecture Overview
# ============================================================================

print("\n📐 ARCHITECTURE COMPARISON")
print("-"*70)

architecture = {
    "Component": ["Entry Point", "Orchestration", "LLM Layer", "Prompt System", "Output Format"],
    "Notebook": ["playground cell", "route_with_intent()", "OpenAI/Anthropic SDK", "Python strings", "JSON parsing"],
    "Production": ["scheduler.py", "talk2bill_pipeline.py", "LangChain+Gemini", "prompts.py constants", "Pydantic models"]
}

df_arch = pd.DataFrame(architecture)
print(df_arch.to_string(index=False))

# ============================================================================
# PART 2: Detailed Feature Comparison
# ============================================================================

print("\n\n📊 FEATURE-BY-FEATURE COMPARISON")
print("-"*70)

features = {
    "Feature": [
        "Primary LLM",
        "Backup LLM",
        "Temperature",
        "Max Tokens",
        "Retry Logic",
        "Config Source",
        "Caching",
        "Categories",
        "Intent Types",
        "Greeting Handling",
        "Multi-turn Conversation",
        "Structured Output",
        "Error Handling",
        "Performance Monitoring",
        "Test Coverage"
    ],
    "Notebooks": [
        "Claude Sonnet 4.5",
        "Claude Haiku 4.5",
        "0.2 (intent), 0.3 (extraction)",
        "200-1000",
        "None",
        "Google Sheets Registry",
        "5-min pickle cache",
        "50 consolidated",
        "7 types",
        "✅ Yes",
        "❌ No (single-turn)",
        "JSON parsing",
        "Basic try-catch",
        "Detailed timing",
        "90 test cases"
    ],
    "Production": [
        "Gemini 2.0 Flash",
        "None",
        "0.0 (all)",
        "Pydantic-defined",
        "3 attempts, 1s delay",
        "Environment variables",
        "None",
        "~10 generic",
        "2 types (expense, other)",
        "❌ No",
        "✅ Yes (with history)",
        "Pydantic models",
        "Retry + logging",
        "Basic logging",
        "Limited"
    ],
    "Priority": [
        "Keep Gemini",
        "Not needed",
        "Keep 0.0",
        "Keep Pydantic",
        "✅ Keep",
        "🟢 Optional",
        "🟢 Optional",
        "🔴 Update",
        "🟡 Expand",
        "🟡 Add",
        "✅ Keep",
        "✅ Keep",
        "🟡 Enhance",
        "🟡 Add",
        "🔴 Add"
    ]
}

df_features = pd.DataFrame(features)
print(df_features.to_string(index=False))

# ============================================================================
# PART 3: Migration Priorities
# ============================================================================

print("\n\n🎯 MIGRATION PRIORITIES BREAKDOWN")
print("-"*70)

priorities = {
    "🔴 HIGH PRIORITY": [
        "Update prompts.py with 50 consolidated categories",
        "Add greeting intent to INTENT_CLASSIFICATION_PROMPT_VYP",
        "Create test suite with 90 test cases from vaani_test_cases.csv",
        "Test category accuracy with Gemini",
        "Update EXPENSE_EXTRACTION_PROMPT_V1 with categorization rules"
    ],
    "🟡 MEDIUM PRIORITY": [
        "Add greeting response handler in talk2bill_pipeline.py",
        "Implement performance monitoring (timing instrumentation)",
        "Enhance JSON parsing robustness in llm_service.py",
        "Add greeting examples to prompts.py",
        "Create benchmark comparison: Gemini vs Claude"
    ],
    "🟢 LOW PRIORITY": [
        "Add optional configuration registry (vyapar_config.py)",
        "Implement caching for repeated queries",
        "Add more detailed logging",
        "Documentation updates",
        "Code refactoring for maintainability"
    ]
}

for priority, items in priorities.items():
    print(f"\n{priority}:")
    for i, item in enumerate(items, 1):
        print(f"  {i}. {item}")

# ============================================================================
# PART 4: Files That Need Updates
# ============================================================================

print("\n\n📁 FILES REQUIRING UPDATES")
print("-"*70)

file_updates = {
    "File": [
        "prompts.py",
        "talk2bill_pipeline.py",
        "llm_service.py",
        "prompt_builder.py",
        "scheduler.py",
        "tests/ (new)",
        "vyapar_config.py (new)"
    ],
    "Priority": [
        "🔴 HIGH",
        "🟡 MEDIUM",
        "🟡 MEDIUM",
        "🟢 LOW",
        "✅ No changes",
        "🔴 HIGH",
        "🟢 LOW (optional)"
    ],
    "Changes": [
        "Add 50 categories, greeting rules/examples",
        "Add greeting response handler",
        "Add timing, improve error handling",
        "Update to use new prompts",
        "No changes needed",
        "Create comprehensive test suite",
        "Optional config centralization"
    ],
    "Lines Changed": [
        "~150 lines",
        "~50 lines",
        "~30 lines",
        "~10 lines",
        "0 lines",
        "~500 lines (new)",
        "~100 lines (new)"
    ]
}

df_updates = pd.DataFrame(file_updates)
print(df_updates.to_string(index=False))

# ============================================================================
# PART 5: Integration Points
# ============================================================================

print("\n\n🔌 KEY INTEGRATION POINTS")
print("-"*70)

print("""
1. LLM SERVICE (llm_service.py):
   - Currently: Uses Gemini 2.0 Flash with LangChain
   - Action: Keep Gemini, just update prompts
   - Testing: Compare Gemini vs Claude accuracy

2. PROMPT TEMPLATES (prompts.py):
   - Currently: ~10 generic categories, no greeting handling
   - Action: Add 50 categories, greeting intent
   - Testing: Validate with 90 test cases

3. PIPELINE ORCHESTRATION (talk2bill_pipeline.py):
   - Currently: 2 intents (expense, other)
   - Action: Add greeting intent handling
   - Testing: Integration tests for new flow

4. CONVERSATION HISTORY:
   - Currently: Multi-turn with 10 message history
   - Action: Keep existing logic (better than notebooks)
   - Testing: Verify greeting doesn't break conversation flow

5. STRUCTURED OUTPUT:
   - Currently: Pydantic models with validation
   - Action: Keep existing approach (better than notebooks)
   - Testing: Ensure new prompts work with existing models
""")

# ============================================================================
# PART 6: Risk Assessment
# ============================================================================

print("\n🚨 RISK ASSESSMENT")
print("-"*70)

risks = {
    "Risk": [
        "Intent accuracy degradation",
        "Category confusion",
        "Greeting loop",
        "Performance regression",
        "Breaking existing flows"
    ],
    "Impact": [
        "HIGH",
        "MEDIUM",
        "LOW",
        "MEDIUM",
        "HIGH"
    ],
    "Probability": [
        "MEDIUM",
        "MEDIUM",
        "LOW",
        "LOW",
        "LOW"
    ],
    "Mitigation": [
        "Extensive testing with 90 cases, shadow deployment",
        "Category accuracy monitoring, user feedback",
        "Simple flow, easy rollback",
        "Performance benchmarking, monitoring",
        "Pydantic validation, comprehensive tests"
    ]
}

df_risks = pd.DataFrame(risks)
print(df_risks.to_string(index=False))

# ============================================================================
# PART 7: Success Metrics
# ============================================================================

print("\n\n✅ SUCCESS CRITERIA")
print("-"*70)

metrics = {
    "Metric": [
        "Intent classification accuracy",
        "Category classification accuracy",
        "Greeting detection accuracy",
        "P95 latency",
        "Error rate",
        "User satisfaction",
        "Expense recording frequency",
        "Voice adoption rate"
    ],
    "Target": [
        "≥ 90%",
        "≥ 85%",
        "≥ 95%",
        "≤ 3000ms",
        "≤ 5%",
        "≥ 4.0/5.0",
        "+20%",
        "+15%"
    ],
    "Measurement": [
        "Test suite accuracy",
        "Sampled transactions",
        "Greeting test cases",
        "Latency monitoring",
        "Error logs",
        "User surveys",
        "Transaction analytics",
        "Feature usage data"
    ]
}

df_metrics = pd.DataFrame(metrics)
print(df_metrics.to_string(index=False))

# ============================================================================
# PART 8: Timeline Summary
# ============================================================================

print("\n\n📅 MIGRATION TIMELINE")
print("-"*70)

timeline = """
Week 1-2:  High Priority Changes
           - Update prompts.py with 50 categories
           - Add greeting intent
           - Create test suite
           - Test category accuracy

Week 3-4:  Medium Priority Changes
           - Add greeting response handler
           - Implement performance monitoring
           - (Optional) Add configuration registry
           - Documentation updates

Week 5:    Testing & Validation
           - Run full test suite
           - Compare Gemini vs. Claude
           - Performance benchmarking
           - Code review

Week 6:    Shadow Deployment
           - Deploy alongside existing
           - Compare outputs
           - Log discrepancies

Week 7:    Canary Release
           - 5% traffic to new version
           - Monitor metrics
           - Collect feedback

Week 8-10: Gradual Rollout
           - 25% → 50% → 75% → 100%
           - Monitor at each stage
           - Final deployment
"""

print(timeline)

# ============================================================================
# PART 9: Action Items
# ============================================================================

print("\n🎯 IMMEDIATE ACTION ITEMS")
print("-"*70)

actions = """
FOR AKHIL:
□ Review migration guide with engineering team
□ Prioritize features based on business impact
□ Set up testing environment
□ Create feature branch for migration work
□ Schedule weekly sync meetings

FOR ENGINEERING:
□ Read VAANI_Production_Migration_Guide.md
□ Set up local development environment
□ Run existing test suite to establish baseline
□ Create test data files from notebooks
□ Begin prompt template updates

FOR DATA SCIENCE:
□ Export 90 test cases CSV
□ Document category mapping logic
□ Prepare benchmarking scripts
□ Set up monitoring dashboards
□ Create performance baseline report
"""

print(actions)

print("\n" + "="*70)
print("📄 Full documentation: VAANI_Production_Migration_Guide.md")
print("="*70)

🏭 VAANI PRODUCTION CODE ANALYSIS

📐 ARCHITECTURE COMPARISON
----------------------------------------------------------------------
    Component             Notebook            Production
  Entry Point      playground cell          scheduler.py
Orchestration  route_with_intent() talk2bill_pipeline.py
    LLM Layer OpenAI/Anthropic SDK      LangChain+Gemini
Prompt System       Python strings  prompts.py constants
Output Format         JSON parsing       Pydantic models


📊 FEATURE-BY-FEATURE COMPARISON
----------------------------------------------------------------------
                Feature                      Notebooks               Production      Priority
            Primary LLM              Claude Sonnet 4.5         Gemini 2.0 Flash   Keep Gemini
             Backup LLM               Claude Haiku 4.5                     None    Not needed
            Temperature 0.2 (intent), 0.3 (extraction)                0.0 (all)      Keep 0.0
             Max Tokens                       